In [4]:
from pyscf import scf,gto
import numpy as np
import inspect
from alch_deriv import alch_deriv
from FcMole import FcM
import matplotlib.pyplot as plt
from pyscf.grad import rhf as grhf  #### very important
from pyscf.hessian import rhf as hrhf # without those two mf.Gradients() and mf.Hessian() don't work
def DeltaV(mol,dL):
    mol.set_rinv_orig_(mol.atom_coords()[0])
    dV=mol.intor('int1e_rinv')*dL[0]
    mol.set_rinv_orig_(mol.atom_coords()[1])
    dV+=mol.intor('int1e_rinv')*dL[1]
    return -dV

The Goal is to show that the mixed derivative can also be computed via:
$$
\frac{\partial}{\partial x} \sum_{ij}P_{ij}\Delta V_{ij} =\sum_{ij} \frac{\partial P_{ij}}{\partial x}\Delta V_{ij}+ P_{ij}\frac{\partial \Delta V_{ij}}{\partial x}
$$

In [5]:
mol1=FcM(fcs=[.001,-.001],atom="H 0 0 0; Li 0 0 2.0",unit="Bohr",basis="STO-3G")
mf1=scf.RHF(mol1)
mf1.scf(dm0=mf1.init_guess_by_atom())
P1=mf1.make_rdm1()
g1=mf1.Gradients()
g1.run()
ge1=g1.grad_elec()

converged SCF energy = -7.78904103070175
--------------- RHF gradients ---------------
         x                y                z
0 H     0.0000000000     0.0000000000     0.2104913783
1 Li     0.0000000000     0.0000000000    -0.2104913783
----------------------------------------------


In [6]:
mol0=gto.M(atom="H 0 0 0; Li 0 0 2.0",unit="Bohr",basis="STO-3G")
mf0=scf.RHF(mol0)
mf0.scf(dm0=mf.init_guess_by_atom())
P0=mf0.make_rdm1()
h=mf0.Hessian()
g0=mf0.Gradients()
g0.run()
ge0=g0.grad_elec()

NameError: name 'mf' is not defined

In [ ]:
# now is to understand how Pyscf computes dP/dx
h1ao=h.make_h1(mf0.mo_coeff,mf0.mo_occ)

In [ ]:
mo_energy = mf0.mo_energy
mo_occ = mf.mo_occ
mo_coeff = mf0.mo_coeff
atmlst = range(mol.natm)

In [ ]:
mo1_a0,mo1_a1=h.solve_mo1(mo_energy,mo_coeff,mo_occ,h1ao)[0]

In [ ]:
mocc = mo_coeff[:,mo_occ>0]

In [ ]:
dm1_1 = np.einsum('ypi,qi->ypq', mo1_a1, mocc)
dm1_1.shape

In [ ]:
dp_cphf=(dm1_1[2]+(dm1_1[2]).T)*2

In [ ]:
dV=DeltaV(mol,[.001,-.001])

In [ ]:
np.einsum('ij,ij',dp_cphf,dV),ge1-ge0

Getting the second part P*dH/dZdx

In [ ]:
NUC_FRAC_CHARGE=gto.mole.NUC_FRAC_CHARGE
NUC_MOD_OF=gto.mole.NUC_MOD_OF
PTR_FRAC_CHARGE=gto.mole.PTR_FRAC_CHARGE
denv=mol._env.copy()
datm=mol._atm.copy()
fcs=[.001,-.001]
datm[:,NUC_MOD_OF] = NUC_FRAC_CHARGE
for i in range (mol.natm):
    denv[datm[i,PTR_FRAC_CHARGE]]=fcs[i] 
dH1=-gto.moleintor.getints('int1e_ipnuc_sph',datm,mol._bas,denv, None,3,0,'s1')

In [ ]:
comp2=np.zeros((2,3,6,6))  
dL=[.001,-.001]
for atm_id in [0,1]:
    with mol.with_rinv_at_nucleus(atm_id):
        vrinv = -mol.intor('int1e_iprinv', comp=3)
    shl0, shl1, p0, p1 = mol.aoslice_by_atom()[atm_id]
    vrinv*=dL[atm_id]
    vrinv[:,p0:p1] += dH1[:,p0:p1] 
    vrinv += vrinv.transpose(0,2,1)
    comp2[atm_id]=vrinv

In [ ]:
np.einsum('xyij,ij->xy',comp2,P) 

In [ ]:
0.00035794-0.00040586+4.766111943088071e-05

In [ ]:
#error<0.1%